# Clustering Crypto

In [152]:
# Initial imports
import pandas as pd
import hvplot.pandas
from holoviews import opts
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [153]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('crypto_data.csv').set_index('Unnamed: 0')
crypto_df.index.name = None
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [154]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading==True]
crypto_df.IsTrading.unique()

array([ True])

In [155]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns='IsTrading', inplace=True)
crypto_df.columns

Index(['CoinName', 'Algorithm', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [156]:
# Remove rows that have at least 1 null value.
for column in crypto_df.columns:
    print(f'Column {column} has {crypto_df[column].isnull().sum()} null values.')

crypto_df.dropna(inplace=True)

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 459 null values.
Column TotalCoinSupply has 0 null values.


In [157]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df.TotalCoinsMined>0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [158]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame(crypto_df['CoinName'])
names_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [159]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns='CoinName', inplace=True)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [160]:
# Use get_dummies() to create variables for text features.
names = pd.get_dummies(crypto_df[['Algorithm','ProofType']])
dummyNames_df = pd.DataFrame(names)
new_crypto_df = crypto_df.drop(['Algorithm','ProofType'], axis=1)
X=pd.concat([dummyNames_df, new_crypto_df], axis=1)
X


,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,Algorithm_Counterparty,...,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,TotalCoinsMined,TotalCoinSupply
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4.199995e+01,42
404,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.055185e+09,532000000
1337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.927942e+10,314159265359
BTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.792718e+07,21000000
ETH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.076842e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.000000e+09,2000000000
GAP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.493105e+07,250000000
BDX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9.802226e+08,1400222610
ZEN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7.296538e+06,21000000


In [161]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [162]:
# Using PCA to reduce dimension to three principal components.

# initialize PCA model with 3 PCs
pca = PCA(n_components=3, random_state=0)

# get 3 PCs for crypto data (X)
pca_X = pca.fit_transform(X)

In [163]:
# Create a DataFrame with the three principal components.

pca_df = pd.DataFrame(
    data = pca_X,
    columns=['PC 1', 'PC 2', 'PC 3'],
    index=crypto_df.index
)

# show sample of pca_df
pca_df.head(10)

,PC 1,PC 2,PC 3
42,-0.344087,1.021689,-0.456908
404,-0.327363,1.021815,-0.457484
1337,2.304783,1.581374,-0.484607
BTC,-0.145258,-1.331324,0.176327
ETH,-0.147878,-2.086784,0.331424
LTC,-0.163725,-1.105191,-0.028736
DASH,-0.415391,1.233715,-0.344841
XMR,-0.142158,-2.223068,0.280411
ETC,-0.146314,-2.086883,0.331393
ZEC,-0.127453,-2.090149,0.372408


In [164]:
# Get explained variance of each PC
pca.explained_variance_ratio_

# not doing a very good job...

array([0.02792532, 0.02140717, 0.0204657 ])

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [165]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [166]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
predictions = model.predict(pca_df)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [167]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
pred_df = pd.DataFrame(predictions, index = crypto_df.index)
pred_df.columns=['Class']

# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pca_df, names_df, pred_df], axis=1)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.344087,1.021689,-0.456908,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327363,1.021815,-0.457484,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.304783,1.581374,-0.484607,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145258,-1.331324,0.176327,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.147878,-2.086784,0.331424,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.163725,-1.105191,-0.028736,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.415391,1.233715,-0.344841,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.142158,-2.223068,0.280411,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146314,-2.086883,0.331393,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.127453,-2.090149,0.372408,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [170]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig =  px.scatter_3d(clustered_df,
x = 'PC 1',
y='PC 2',
z = 'PC 3',
color='Class',
hover_name = 'CoinName',
hover_data= ['Algorithm'])

# show plot
fig.show()

In [174]:
# Create a table with tradeable cryptocurrencies.
clustered_df.hvplot.table(
    columns=['CoinName',
    'Algorithm',
    'ProofType',
    'TotalCoinSupply',
    'TotalCoinsMined',
    'Class'
    ],
    sortable=True,
    selectable = True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [ ]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
